In [1]:


!pip install flask flask-cors flask-sqlalchemy flask-socketio python-dotenv nltk pandas pyngrok langdetect textblob deep-translator eventlet -q

import nltk
print("📥 Downloading NLTK data...")
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('brown', quiet=True)

print("✅ All dependencies installed successfully!")
print("✅ NLTK data downloaded!")
print("✅ Eventlet installed for SocketIO!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.3/364.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 1.9 MB/s eta 0:00:00
📥 Downloading NLTK data...
✅ All dependencies installed successfully!
✅ NLTK data downloaded!
✅ Eventlet installed for SocketIO!


In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from flask_sqlalchemy import SQLAlchemy
from flask_socketio import SocketIO, emit, join_room
from datetime import datetime
from collections import Counter
from deep_translator import GoogleTranslator
from langdetect import detect, DetectorFactory
import json
import os
import re
from pyngrok import ngrok

# NLTK imports
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from textblob import TextBlob


DetectorFactory.seed = 0

print("✅ All libraries imported successfully!")
print("✅ Using deep-translator (no httpx conflicts!)")

✅ All libraries imported successfully!
✅ Using deep-translator (no httpx conflicts!)


In [3]:

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///travel_planner.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'your-secret-key-for-colab'

db = SQLAlchemy(app)

CORS(app, resources={
    r"/*": {
        "origins": "*",
        "methods": ["GET", "POST", "PUT", "DELETE", "OPTIONS"],
        "allow_headers": ["Content-Type", "Authorization", "ngrok-skip-browser-warning"],
        "expose_headers": ["Content-Type"],
        "supports_credentials": False
    }
})



@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization,ngrok-skip-browser-warning')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    return response

print("✅ Flask app initialized (REST API mode)!")

✅ Flask app initialized (REST API mode)!


In [4]:
class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(100), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class Preference(db.Model):
    __tablename__ = 'preferences'
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    group_id = db.Column(db.Integer, db.ForeignKey('groups.id'))
    preference_text = db.Column(db.Text, nullable=False)
    original_language = db.Column(db.String(10))
    translated_text = db.Column(db.Text)
    extracted_categories = db.Column(db.JSON)
    extracted_keywords = db.Column(db.JSON)
    sentiment_score = db.Column(db.Float)
    budget = db.Column(db.String(20))
    duration_days = db.Column(db.Integer)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class Group(db.Model):
    __tablename__ = 'groups'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(200), nullable=False)
    created_by = db.Column(db.Integer, db.ForeignKey('users.id'))
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class GroupMember(db.Model):
    __tablename__ = 'group_members'
    id = db.Column(db.Integer, primary_key=True)
    group_id = db.Column(db.Integer, db.ForeignKey('groups.id'), nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    joined_at = db.Column(db.DateTime, default=datetime.utcnow)

class Attraction(db.Model):
    __tablename__ = 'attractions'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(200), nullable=False)
    location = db.Column(db.String(200))
    state = db.Column(db.String(100))
    category = db.Column(db.String(50))
    description = db.Column(db.Text)
    rating = db.Column(db.Float)
    estimated_time_hours = db.Column(db.Float)
    best_time_of_day = db.Column(db.String(20))
    entry_fee = db.Column(db.Float)

class Itinerary(db.Model):
    __tablename__ = 'itineraries'
    id = db.Column(db.Integer, primary_key=True)
    group_id = db.Column(db.Integer, db.ForeignKey('groups.id'), nullable=False)
    name = db.Column(db.String(200))
    total_days = db.Column(db.Integer)
    generated_plan = db.Column(db.JSON)
    status = db.Column(db.String(20), default='draft')
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

class Expense(db.Model):
    __tablename__ = 'expenses'
    id = db.Column(db.Integer, primary_key=True)
    group_id = db.Column(db.Integer, db.ForeignKey('groups.id'), nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    description = db.Column(db.String(300), nullable=False)
    amount = db.Column(db.Float, nullable=False)
    category = db.Column(db.String(50), nullable=False)
    paid_by = db.Column(db.Integer, db.ForeignKey('users.id'), nullable=False)
    split_among = db.Column(db.JSON)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

print("✅ Database models defined!")

✅ Database models defined!


In [5]:


class EnhancedLanguageProcessor:
    def __init__(self):
        self.translator = GoogleTranslator()
        self.lemmatizer = WordNetLemmatizer()

        self.supported_languages = {
            'en': 'English',
            'hi': 'Hindi (हिंदी)',
            'bn': 'Bengali (বাংলা)',
            'ta': 'Tamil (தமிழ்)',
            'te': 'Telugu (తెలుగు)',
            'ml': 'Malayalam (മലയാളം)',
            'mr': 'Marathi (मराठी)',
            'gu': 'Gujarati (ગુજરાતી)',
            'kn': 'Kannada (ಕನ್ನಡ)',
            'pa': 'Punjabi (ਪੰਜਾਬੀ)',
            'or': 'Odia (ଓଡ଼ିଆ)',
            'as': 'Assamese (অসমীয়া)',
            'ur': 'Urdu (اردو)'
        }

        self.english_stopwords = set(stopwords.words('english'))

    def detect_language(self, text):
        """Detect language using langdetect"""
        try:
            detected = detect(text)
            lang_name = self.supported_languages.get(detected, f'Unknown ({detected})')
            print(f"🌐 Language detected: {detected} - {lang_name}")
            return detected
        except Exception as e:
            print(f"⚠️ Detection failed: {e}, defaulting to English")
            return 'en'

    def tokenize_text(self, text):
        """Tokenize using NLTK - fallback to simple split if punkt fails"""
        try:
            tokens = word_tokenize(text.lower())
            print(f"📝 NLTK Tokens (first 15): {tokens[:15]}")
            return tokens
        except Exception as e:
            print(f"⚠️ NLTK tokenization failed, using simple split")
            tokens = text.lower().split()
            print(f"📝 Simple Tokens (first 15): {tokens[:15]}")
            return tokens

    def remove_stopwords(self, tokens):
        """Remove stopwords using NLTK"""
        filtered = [word for word in tokens if word.isalnum() and word not in self.english_stopwords]
        print(f"🧹 Stopword removal: {len(tokens)} → {len(filtered)} tokens")
        return filtered

    def lemmatize_tokens(self, tokens):
        """Lemmatize tokens using NLTK WordNet"""
        lemmatized = [self.lemmatizer.lemmatize(token) for token in tokens]
        print(f"🔤 Lemmatized (first 10): {lemmatized[:10]}")
        return lemmatized

    def extract_pos_tags(self, text):
        """Extract POS tags and categorize words - with fallback"""
        try:
            tokens = word_tokenize(text)
        except:
            print("⚠️ Using simple tokenization for POS tagging")
            tokens = text.split()

        try:
            pos_tags = pos_tag(tokens)

            nouns = [word for word, pos in pos_tags if pos.startswith('NN')]
            verbs = [word for word, pos in pos_tags if pos.startswith('VB')]
            adjectives = [word for word, pos in pos_tags if pos.startswith('JJ')]

            print(f"🏷️ POS Analysis:")
            print(f"   Nouns: {nouns[:8]}")
            print(f"   Verbs: {verbs[:5]}")
            print(f"   Adjectives: {adjectives[:5]}")

            return {
                'nouns': nouns,
                'verbs': verbs,
                'adjectives': adjectives,
                'all_tags': pos_tags
            }
        except Exception as e:
            print(f"⚠️ POS tagging error: {e}")
            return {
                'nouns': [],
                'verbs': [],
                'adjectives': [],
                'all_tags': []
            }

    def analyze_sentiment(self, text):
        """Analyze sentiment using TextBlob"""
        try:
            blob = TextBlob(text)
            sentiment = blob.sentiment.polarity

            if sentiment > 0.1:
                sentiment_label = "Positive 😊"
            elif sentiment < -0.1:
                sentiment_label = "Negative 😟"
            else:
                sentiment_label = "Neutral 😐"

            print(f"😊 Sentiment: {sentiment_label} (score: {sentiment:.2f})")
            return sentiment
        except:
            return 0.0

    def translate_to_english(self, text, source_lang):
        """Translate to English using Google Translate"""
        if source_lang == 'en':
            return text

        try:
            translated = GoogleTranslator(source=source_lang, target='en').translate(text)
            print(f"🔄 Translation:")
            print(f"   Original ({source_lang}): {text}")
            print(f"   English: {translated}")
            return translated
        except Exception as e:
            print(f"⚠️ Translation error: {e}")
            return text


class EnhancedPreferenceExtractor:
    def __init__(self):
        self.category_keywords = {
            'religious': [
                'temple', 'church', 'mosque', 'gurudwara', 'shrine', 'monastery',
                'spiritual', 'pilgrimage', 'worship', 'deity', 'god', 'goddess', 'prayer',
                'मंदिर', 'मस्जिद', 'चर्च', 'গির্জা', 'মসজিদ', 'கோயில்', 'దేవాలయం',
                'ക്ഷേത്രം', 'ದೇವಾಲಯ', 'મંદિર', 'ਗੁਰਦੁਆਰਾ', 'ମନ୍ଦିର'
            ],
            'beach': [
                'beach', 'sea', 'ocean', 'coast', 'shore', 'island', 'sand', 'wave', 'surf',
                'समुद्र', 'तट', 'সমুদ্র', 'সৈকত', 'கடற்கரை', 'సముద్రం', 'കടൽ', 'ಸಮುದ್ರ',
                'સમુદ્ર', 'ਸਮੁੰਦਰ', 'ସମୁଦ୍ର'
            ],
            'historical': [
                'historical', 'monument', 'fort', 'palace', 'heritage', 'ancient',
                'ruins', 'museum', 'castle', 'archaeological', 'historic',
                'किला', 'महल', 'ऐतिहासिक', 'কিল্লা', 'প্রাসাদ', 'கோட்டை', 'కోట',
                'കോട്ട', 'ಕೋಟೆ', 'કિલ્લો', 'ਕਿਲ੍ਹਾ', 'ଦୁର୍ଗ'
            ],
            'nature': [
                'nature', 'park', 'wildlife', 'forest', 'mountain', 'hill', 'valley',
                'waterfall', 'lake', 'garden', 'trek', 'hiking', 'camping', 'natural',
                'प्रकृति', 'पार्क', 'জঙ্গল', 'প্রকৃতি', 'இயற்கை', 'ప్రకృతి',
                'പ്രകൃതി', 'ಪ್ರಕೃತಿ', 'પ્રકૃતિ', 'ਕੁਦਰਤ', 'ପ୍ରକୃତି'
            ],
            'adventure': [
                'adventure', 'trekking', 'rafting', 'paragliding', 'camping',
                'rock climbing', 'skiing', 'diving', 'bungee', 'zip line', 'safari',
                'साहसिक', 'অ্যাডভেঞ্চার', 'சாகசம்', 'సాహసం', 'സാഹസികത',
                'ಸಾಹಸ', 'સાહસ', 'ਐਡਵੈਂਚਰ'
            ],
            'cultural': [
                'cultural', 'art', 'dance', 'music', 'festival', 'local',
                'traditional', 'handicraft', 'folk', 'culture',
                'सांस्कृतिक', 'संस्कृति', 'সাংস্কৃতিক', 'கலாசாரம்', 'సాంస్కృతిక',
                'സംസ്കാരം', 'ಸಂಸ್ಕೃತಿ', 'સંસ્કૃતિ', 'ਸੱਭਿਆਚਾਰ'
            ],
            'food': [
                'food', 'cuisine', 'restaurant', 'street food', 'culinary', 'dish',
                'meal', 'eating', 'dining', 'taste', 'delicious',
                'खाना', 'भोजन', 'খাবার', 'உணவு', 'ఆహారం', 'ഭക്ഷണം', 'ಆಹಾರ',
                'ખોરાક', 'ਭੋਜਨ', 'ଖାଦ୍ୟ'
            ],
            'shopping': [
                'shopping', 'market', 'bazaar', 'mall', 'shop', 'buy', 'purchase',
                'बाजार', 'खरीदारी', 'বাজার', 'சந்தை', 'మార్కెట్', 'ഷോപ്പിംഗ്',
                'ಶಾಪಿಂಗ್', 'બજાર', 'ਬਾਜ਼ਾਰ'
            ]
        }

        self.budget_keywords = {
            'low': [
                'budget', 'cheap', 'affordable', 'economical', 'inexpensive', 'low cost',
                'सस्ता', 'কম দাম', 'மலிவு', 'చౌకైన', 'വിലകുറഞ്ഞ', 'ಅಗ್ಗದ', 'સસ્તું', 'ਸਸਤਾ'
            ],
            'medium': [
                'moderate', 'medium', 'reasonable', 'average', 'mid range',
                'मध्यम', 'মাঝারি', 'நடுத்தர', 'మధ్యస్థ', 'മദ്ധ്യം', 'ಮಧ್ಯಮ', 'મધ્યમ', 'ਮੱਧਮ'
            ],
            'high': [
                'luxury', 'premium', 'expensive', 'high end', 'upscale', 'lavish',
                'महंगा', 'দামি', 'விலை உயர்ந்த', 'ఖరీదైన', 'വിലയേറിയ', 'ದುಬಾರಿ', 'મોંઘું', 'ਮਹਿੰਗਾ'
            ]
        }

    def extract_categories(self, text, tokens):
        """Extract categories from text and tokens"""
        text_lower = text.lower()
        found_categories = set()

        for category, keywords in self.category_keywords.items():
            for keyword in keywords:
                if keyword in text_lower:
                    found_categories.add(category)
                    break

        for category, keywords in self.category_keywords.items():
            for token in tokens:
                if token in keywords or any(kw in token for kw in keywords if len(kw) > 3):
                    found_categories.add(category)
                    break

        result = list(found_categories) if found_categories else ['general']
        print(f"🏷️ Categories extracted: {result}")
        return result

    def extract_budget(self, text):
        """Extract budget level"""
        text_lower = text.lower()

        for budget_level, keywords in self.budget_keywords.items():
            for keyword in keywords:
                if keyword in text_lower:
                    print(f"💰 Budget detected (keyword): {budget_level}")
                    return budget_level

        numbers = re.findall(r'\d+', text)
        if numbers:
            budget_amount = int(numbers[0])
            if budget_amount < 10000:
                budget = 'low'
            elif budget_amount < 50000:
                budget = 'medium'
            else:
                budget = 'high'
            print(f"💰 Budget detected (amount): {budget} (₹{budget_amount})")
            return budget

        print("💰 Budget defaulted: medium")
        return 'medium'

    def extract_duration(self, text):
        """Extract trip duration"""
        patterns = [
            r'(\d+)\s*days?', r'(\d+)\s*day', r'(\d+)\s*दिन', r'(\d+)\s*দিন',
            r'(\d+)\s*நாள்', r'(\d+)\s*రోజులు', r'(\d+)\s*ദിവസം', r'(\d+)\s*ದಿನ',
            r'(\d+)\s*દિવસ', r'(\d+)\s*ਦਿਨ', r'(\d+)\s*ଦିନ'
        ]

        for pattern in patterns:
            match = re.search(pattern, text.lower())
            if match:
                duration = int(match.group(1))
                print(f"📅 Duration extracted: {duration} days")
                return duration

        print("📅 Duration defaulted: 3 days")
        return 3

    def extract_keywords(self, tokens, pos_tags):
        """Extract important keywords"""
        important = []
        for word, tag in pos_tags:
            if (tag.startswith('NN') or tag.startswith('VB') or tag.startswith('JJ')) and len(word) > 3:
                important.append(word.lower())

        keywords = list(set(important))[:15]
        print(f"🔑 Keywords extracted: {keywords}")
        return keywords


class ItineraryAggregator:
    def aggregate_preferences(self, preferences):
        all_categories = []
        budgets = []
        durations = []

        for pref in preferences:
            if pref.extracted_categories:
                all_categories.extend(pref.extracted_categories)
            if pref.budget:
                budgets.append(pref.budget)
            if pref.duration_days:
                durations.append(pref.duration_days)

        category_counts = Counter(all_categories)
        budget_mode = Counter(budgets).most_common(1)[0][0] if budgets else 'medium'
        avg_duration = sum(durations) // len(durations) if durations else 3

        return {
            'categories': dict(category_counts),
            'budget': budget_mode,
            'duration': avg_duration
        }

    def generate_itinerary(self, aggregated, attractions):
        duration = aggregated['duration']
        categories = aggregated['categories']
        sorted_categories = sorted(categories.items(), key=lambda x: x[1], reverse=True)

        itinerary = []
        used_attractions = set()

        for day in range(1, duration + 1):
            day_plan = {
                'day': day,
                'activities': {'morning': None, 'afternoon': None, 'evening': None}
            }

            for time_slot in ['morning', 'afternoon', 'evening']:
                if sorted_categories:
                    category_idx = ((day - 1) * 3 + ['morning', 'afternoon', 'evening'].index(time_slot)) % len(sorted_categories)
                    target_category = sorted_categories[category_idx][0]

                    suitable_attractions = [
                        a for a in attractions
                        if a.category == target_category and a.id not in used_attractions
                    ]

                    if suitable_attractions:
                        selected = suitable_attractions[0]
                        day_plan['activities'][time_slot] = {
                            'attraction_id': selected.id,
                            'name': selected.name,
                            'location': selected.location,
                            'category': selected.category,
                            'estimated_time': selected.estimated_time_hours
                        }
                        used_attractions.add(selected.id)

            itinerary.append(day_plan)

        return itinerary


class SmartItineraryGenerator:
    """
    Intelligent itinerary generator that creates geographically sensible plans
    """

    def __init__(self):
        # Group Indian attractions by region for travel feasibility
        self.regions = {
            'North': ['Delhi', 'Agra', 'Jaipur', 'Amritsar', 'Katra', 'Rishikesh', 'Nainital', 'Chamoli', 'Bir'],
            'South': ['Madurai', 'Hampi', 'Kerala', 'Alleppey', 'Mysore', 'Chennai', 'Kottayam'],
            'West': ['Goa', 'South Goa', 'North Goa', 'Old Goa', 'Mumbai'],
            'East': ['Kolkata'],
            'Islands': ['Havelock Island', 'Andaman'],
            'Rajasthan': ['Jaipur', 'Rajasthan']
        }

    def get_region(self, location):
        """Identify which region an attraction belongs to"""
        for region, locations in self.regions.items():
            for loc in locations:
                if loc.lower() in location.lower():
                    return region
        return 'Other'

    def generate_smart_itinerary(self, aggregated, attractions):
        """
        Generate a geographically sensible itinerary
        """
        duration = aggregated['duration']
        categories = aggregated['categories']

        # Sort categories by user preference
        sorted_categories = sorted(categories.items(), key=lambda x: x[1], reverse=True)
        preferred_categories = [cat[0] for cat in sorted_categories]

        # Filter attractions by preferred categories
        relevant_attractions = [
            a for a in attractions
            if a.category in preferred_categories
        ]

        # Sort by rating (highest first)
        relevant_attractions.sort(key=lambda x: x.rating if x.rating else 0, reverse=True)

        # Group attractions by region
        attractions_by_region = {}
        for attr in relevant_attractions:
            region = self.get_region(attr.location)
            if region not in attractions_by_region:
                attractions_by_region[region] = []
            attractions_by_region[region].append(attr)

        # Select the best region
        if not attractions_by_region:
            return self._generate_fallback_itinerary(duration)

        # Pick region with most attractions
        best_region = max(attractions_by_region.items(), key=lambda x: len(x[1]))[0]
        region_attractions = attractions_by_region[best_region]

        # Generate day-by-day plan
        itinerary = []
        used_attractions = set()

        for day in range(1, duration + 1):
            day_plan = {
                'day': day,
                'region': best_region,
                'activities': {
                    'morning': None,
                    'afternoon': None,
                    'evening': None
                }
            }

            time_slots = ['morning', 'afternoon', 'evening']

            for slot in time_slots:
                for attr in region_attractions:
                    if attr.id not in used_attractions:
                        if attr.best_time_of_day == slot or attr.best_time_of_day is None:
                            day_plan['activities'][slot] = {
                                'attraction_id': attr.id,
                                'name': attr.name,
                                'location': attr.location,
                                'state': attr.state,
                                'category': attr.category,
                                'rating': attr.rating,
                                'estimated_time': attr.estimated_time_hours,
                                'entry_fee': attr.entry_fee,
                                'description': attr.description
                            }
                            used_attractions.add(attr.id)
                            break

                if day_plan['activities'][slot] is None:
                    for attr in region_attractions:
                        if attr.id not in used_attractions:
                            day_plan['activities'][slot] = {
                                'attraction_id': attr.id,
                                'name': attr.name,
                                'location': attr.location,
                                'state': attr.state,
                                'category': attr.category,
                                'rating': attr.rating,
                                'estimated_time': attr.estimated_time_hours,
                                'entry_fee': attr.entry_fee,
                                'description': attr.description
                            }
                            used_attractions.add(attr.id)
                            break

            itinerary.append(day_plan)

        return itinerary

    def _generate_fallback_itinerary(self, duration):
        """Fallback for when no attractions match"""
        itinerary = []
        for day in range(1, duration + 1):
            itinerary.append({
                'day': day,
                'region': 'General',
                'activities': {
                    'morning': None,
                    'afternoon': None,
                    'evening': None
                }
            })
        return itinerary


# Initialize NLP components
lang_processor = EnhancedLanguageProcessor()
pref_extractor = EnhancedPreferenceExtractor()
aggregator = ItineraryAggregator()
smart_generator = SmartItineraryGenerator()

print("✅ Enhanced NLP with NLTK initialized!")
print("✅ Smart Itinerary Generator initialized!")
print("✅ Using deep-translator (no conflicts!)")
print("📚 NLP Features: Tokenization, POS Tagging, Lemmatization, Sentiment Analysis")
print("📍 Smart routing: Geographically sensible itineraries")

✅ Enhanced NLP with NLTK initialized!
✅ Smart Itinerary Generator initialized!
✅ Using deep-translator (no conflicts!)
📚 NLP Features: Tokenization, POS Tagging, Lemmatization, Sentiment Analysis
📍 Smart routing: Geographically sensible itineraries


In [6]:
# Check if routes are already registered, if so skip all route definitions
if 'home' in app.view_functions:
    print("⚠️ Routes already registered. Skipping...")
else:
    @app.route('/')
    def home():
        return jsonify({
            'message': 'Enhanced Travel Planner with NLTK',
            'status': 'success',
            'nlp_features': [
                'NLTK Tokenization',
                'POS Tagging (Nouns, Verbs, Adjectives)',
                'Lemmatization',
                'Stopword Removal',
                'Sentiment Analysis',
                'Multi-language Support (13 Indian languages)'
            ]
        })

    @app.route('/api/languages', methods=['GET'])
    def get_supported_languages():
        return jsonify({
            'languages': lang_processor.supported_languages,
            'count': len(lang_processor.supported_languages)
        })

    @app.route('/api/users', methods=['POST'])
    def create_user():
        try:
            data = request.json
            existing_user = User.query.filter_by(email=data['email']).first()
            if existing_user:
                return jsonify({
                    'id': existing_user.id,
                    'username': existing_user.username,
                    'message': 'User logged in!'
                }), 200

            user = User(username=data['username'], email=data['email'])
            db.session.add(user)
            db.session.commit()
            return jsonify({
                'id': user.id,
                'username': user.username,
                'message': 'User created!'
            }), 201
        except Exception as e:
            db.session.rollback()
            return jsonify({'error': str(e)}), 400

    @app.route('/api/preferences', methods=['POST'])
    def submit_preference():
        try:
            data = request.json
            original_text = data['preference_text']

            print("\n" + "="*70)
            print("🔍 NLTK-POWERED NLP PROCESSING")
            print("="*70)

            # Step 1: Detect language
            detected_lang = lang_processor.detect_language(original_text)

            # Step 2: Translate to English
            english_text = lang_processor.translate_to_english(original_text, detected_lang)

            # Step 3: Tokenize
            tokens = lang_processor.tokenize_text(english_text)

            # Step 4: Remove stopwords
            filtered_tokens = lang_processor.remove_stopwords(tokens)

            # Step 5: Lemmatize
            lemmatized = lang_processor.lemmatize_tokens(filtered_tokens)

            # Step 6: POS tagging
            pos_info = lang_processor.extract_pos_tags(english_text)

            # Step 7: Sentiment analysis
            sentiment = lang_processor.analyze_sentiment(english_text)

            # Step 8: Extract categories
            categories = pref_extractor.extract_categories(english_text, lemmatized)

            # Step 9: Extract budget
            budget = pref_extractor.extract_budget(english_text)

            # Step 10: Extract duration
            duration = pref_extractor.extract_duration(english_text)

            # Step 11: Extract keywords
            keywords = pref_extractor.extract_keywords(lemmatized, pos_info['all_tags'])

            print("="*70)
            print("✅ NLP PROCESSING COMPLETE")
            print("="*70 + "\n")

            # Save preference
            preference = Preference(
                user_id=data['user_id'],
                group_id=data.get('group_id'),
                preference_text=original_text,
                original_language=detected_lang,
                translated_text=english_text,
                extracted_categories=categories,
                extracted_keywords=keywords,
                sentiment_score=sentiment,
                budget=budget,
                duration_days=duration
            )
            db.session.add(preference)
            db.session.commit()

            return jsonify({
                'id': preference.id,
                'original_language': detected_lang,
                'language_name': lang_processor.supported_languages.get(detected_lang, 'Unknown'),
                'original_text': original_text,
                'translated_text': english_text,
                'categories': categories,
                'keywords': keywords,
                'nouns_found': pos_info['nouns'][:5],
                'verbs_found': pos_info['verbs'][:5],
                'adjectives_found': pos_info['adjectives'][:5],
                'sentiment': sentiment,
                'budget': budget,
                'duration': duration,
                'nlp_engine': 'NLTK + TextBlob'
            }), 201
        except Exception as e:
            db.session.rollback()
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()
            return jsonify({'error': str(e)}), 400

    @app.route('/api/groups', methods=['POST'])
    def create_group():
        try:
            data = request.json
            group = Group(name=data['name'], created_by=data['user_id'])
            db.session.add(group)
            db.session.commit()

            member = GroupMember(group_id=group.id, user_id=data['user_id'])
            db.session.add(member)
            db.session.commit()

            return jsonify({'id': group.id, 'name': group.name}), 201
        except Exception as e:
            db.session.rollback()
            return jsonify({'error': str(e)}), 400

    @app.route('/api/groups/<int:group_id>/itinerary', methods=['POST'])
    def generate_itinerary(group_id):
        try:
            preferences = Preference.query.filter_by(group_id=group_id).all()

            if not preferences:
                return jsonify({'error': 'No preferences found for this group'}), 400

            # Aggregate preferences
            aggregated = aggregator.aggregate_preferences(preferences)

            # Get all attractions
            attractions = Attraction.query.all()

            # ✅ CHANGED: Use smart generator instead of basic one
            itinerary_plan = smart_generator.generate_smart_itinerary(aggregated, attractions)

            # Calculate trip summary
            total_cost = 0
            attractions_count = 0

            for day in itinerary_plan:
                for slot in ['morning', 'afternoon', 'evening']:
                    activity = day['activities'][slot]
                    if activity:
                        total_cost += activity.get('entry_fee', 0)
                        attractions_count += 1

            # Save itinerary
            itinerary = Itinerary(
                group_id=group_id,
                name=f"{itinerary_plan[0]['region'] if itinerary_plan else 'India'} Trip - {aggregated['duration']} days",
                total_days=aggregated['duration'],
                generated_plan=itinerary_plan
            )
            db.session.add(itinerary)
            db.session.commit()

            return jsonify({
                'id': itinerary.id,
                'plan': itinerary_plan,
                'summary': {
                    'region': itinerary_plan[0]['region'] if itinerary_plan else 'General',
                    'total_days': aggregated['duration'],
                    'total_attractions': attractions_count,
                    'estimated_entry_fees': total_cost,
                    'budget_level': aggregated['budget']
                }
            }), 201

        except Exception as e:
            db.session.rollback()
            print(f"❌ Error generating itinerary: {e}")
            import traceback
            traceback.print_exc()
            return jsonify({'error': str(e)}), 400

    @app.route('/api/itineraries/<int:itinerary_id>', methods=['GET'])
    def get_itinerary(itinerary_id):
        try:
            itinerary = Itinerary.query.get_or_404(itinerary_id)
            return jsonify({
                'id': itinerary.id,
                'name': itinerary.name,
                'total_days': itinerary.total_days,
                'generated_plan': itinerary.generated_plan,
                'status': itinerary.status
            })
        except Exception as e:
            return jsonify({'error': str(e)}), 400

    @app.route('/api/attractions', methods=['GET'])
    def get_attractions():
        try:
            attractions = Attraction.query.all()
            return jsonify([{
                'id': a.id, 'name': a.name, 'location': a.location,
                'category': a.category, 'rating': a.rating
            } for a in attractions])
        except Exception as e:
            return jsonify({'error': str(e)}), 400

    @app.route('/api/groups/<int:group_id>', methods=['GET'])
    def get_group(group_id):
        try:
            group = Group.query.get_or_404(group_id)
            members = db.session.query(User).join(GroupMember).filter(GroupMember.group_id == group_id).all()

            return jsonify({
                'id': group.id,
                'name': group.name,
                'members': [{'id': m.id, 'username': m.username} for m in members]
            })
        except Exception as e:
            return jsonify({'error': str(e)}), 400

    print("✅ API routes defined with NLTK processing!")

✅ API routes defined with NLTK processing!


In [7]:
import os

# Force delete old database
db_path = 'travel_planner.db'
if os.path.exists(db_path):
    try:
        db.session.remove()
        db.engine.dispose()
    except:
        pass
    os.remove(db_path)
    print(f"🗑️ Deleted old database")

# Create new database
with app.app_context():
    db.create_all()
    print("✅ Database created!")

    # Verify schema
    from sqlalchemy import inspect
    inspector = inspect(db.engine)
    columns = [col['name'] for col in inspector.get_columns('preferences')]
    print(f"📋 Preference columns: {columns}")

    if 'sentiment_score' in columns:
        print("✅ New schema confirmed with sentiment_score!")

    # Seed attractions
    if Attraction.query.count() == 0:
        attractions_data = [
            # Religious Sites
            {'name': 'Golden Temple', 'location': 'Amritsar', 'state': 'Punjab', 'category': 'religious',
             'description': 'Holiest Gurudwara of Sikhs', 'rating': 4.9, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'morning', 'entry_fee': 0},
            {'name': 'Meenakshi Temple', 'location': 'Madurai', 'state': 'Tamil Nadu', 'category': 'religious',
             'description': 'Historic Hindu temple', 'rating': 4.8, 'estimated_time_hours': 2.5,
             'best_time_of_day': 'morning', 'entry_fee': 50},
            {'name': 'Vaishno Devi', 'location': 'Katra', 'state': 'J&K', 'category': 'religious',
             'description': 'Famous pilgrimage', 'rating': 4.8, 'estimated_time_hours': 6.0,
             'best_time_of_day': 'morning', 'entry_fee': 0},

            # Beaches
            {'name': 'Radhanagar Beach', 'location': 'Havelock Island', 'state': 'Andaman', 'category': 'beach',
             'description': 'Pristine beach', 'rating': 4.7, 'estimated_time_hours': 4.0,
             'best_time_of_day': 'afternoon', 'entry_fee': 0},
            {'name': 'Palolem Beach', 'location': 'South Goa', 'state': 'Goa', 'category': 'beach',
             'description': 'Scenic beach', 'rating': 4.6, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'evening', 'entry_fee': 0},
            {'name': 'Kovalam Beach', 'location': 'Kerala', 'state': 'Kerala', 'category': 'beach',
             'description': 'Lighthouse beach', 'rating': 4.5, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'afternoon', 'entry_fee': 0},

            # Historical
            {'name': 'Taj Mahal', 'location': 'Agra', 'state': 'UP', 'category': 'historical',
             'description': 'Iconic monument', 'rating': 4.9, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'morning', 'entry_fee': 250},
            {'name': 'Amber Fort', 'location': 'Jaipur', 'state': 'Rajasthan', 'category': 'historical',
             'description': 'Hill fort', 'rating': 4.7, 'estimated_time_hours': 3.5,
             'best_time_of_day': 'afternoon', 'entry_fee': 200},
            {'name': 'Hampi Ruins', 'location': 'Hampi', 'state': 'Karnataka', 'category': 'historical',
             'description': 'UNESCO site', 'rating': 4.8, 'estimated_time_hours': 4.0,
             'best_time_of_day': 'morning', 'entry_fee': 40},
            {'name': 'Red Fort', 'location': 'Delhi', 'state': 'Delhi', 'category': 'historical',
             'description': 'Mughal fort', 'rating': 4.6, 'estimated_time_hours': 2.5,
             'best_time_of_day': 'afternoon', 'entry_fee': 35},

            # Nature
            {'name': 'Valley of Flowers', 'location': 'Chamoli', 'state': 'Uttarakhand', 'category': 'nature',
             'description': 'Alpine sanctuary', 'rating': 4.8, 'estimated_time_hours': 5.0,
             'best_time_of_day': 'morning', 'entry_fee': 150},
            {'name': 'Kerala Backwaters', 'location': 'Alleppey', 'state': 'Kerala', 'category': 'nature',
             'description': 'Serene waterways', 'rating': 4.7, 'estimated_time_hours': 5.0,
             'best_time_of_day': 'afternoon', 'entry_fee': 0},
            {'name': 'Jim Corbett Park', 'location': 'Nainital', 'state': 'Uttarakhand', 'category': 'nature',
             'description': 'Tiger reserve', 'rating': 4.7, 'estimated_time_hours': 4.0,
             'best_time_of_day': 'morning', 'entry_fee': 600},

            # Adventure
            {'name': 'Rishikesh Rafting', 'location': 'Rishikesh', 'state': 'Uttarakhand', 'category': 'adventure',
             'description': 'River rafting', 'rating': 4.8, 'estimated_time_hours': 4.0,
             'best_time_of_day': 'morning', 'entry_fee': 500},
            {'name': 'Bir Billing', 'location': 'Bir', 'state': 'HP', 'category': 'adventure',
             'description': 'Paragliding', 'rating': 4.7, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'afternoon', 'entry_fee': 2500},

            # Cultural
            {'name': 'City Palace Jaipur', 'location': 'Jaipur', 'state': 'Rajasthan', 'category': 'cultural',
             'description': 'Royal palace', 'rating': 4.6, 'estimated_time_hours': 2.5,
             'best_time_of_day': 'afternoon', 'entry_fee': 200},

            # Food
            {'name': 'Old Delhi Food Walk', 'location': 'Delhi', 'state': 'Delhi', 'category': 'food',
             'description': 'Street food tour', 'rating': 4.8, 'estimated_time_hours': 3.0,
             'best_time_of_day': 'evening', 'entry_fee': 500},
        ]

        for data in attractions_data:
            attraction = Attraction(**data)
            db.session.add(attraction)

        db.session.commit()
        print(f"✅ Seeded {len(attractions_data)} attractions!")

print("\n✅ Database ready!")


✅ Database created!
📋 Preference columns: ['id', 'user_id', 'group_id', 'preference_text', 'original_language', 'translated_text', 'extracted_categories', 'extracted_keywords', 'sentiment_score', 'budget', 'duration_days', 'created_at']
✅ New schema confirmed with sentiment_score!
✅ Seeded 17 attractions!

✅ Database ready!


In [ ]:
import time
from pyngrok import ngrok

# Set your ngrok auth token
ngrok.set_auth_token("35o5ALOM4iWSujVlrjCWcW4BVfc_2dUYJxYdbNeSq5GQuZXTy")

# Clean up
print("🔄 Cleaning up...")
try:
    ngrok.kill()
    time.sleep(2)
except:
    pass

# Start tunnel
try:
    print("🚀 Starting NLTK-enhanced server...")

    public_url = ngrok.connect(5000)

    # Extract clean URL
    url_str = str(public_url)
    if '"' in url_str:
        clean_url = url_str.split('"')[1]
    elif "->" in url_str:
        clean_url = url_str.split("->")[0].strip()
    else:
        clean_url = url_str

    print(f"\n{'='*70}")
    print(f"✅ NLTK-POWERED TRAVEL PLANNER IS LIVE!")
    print(f"{'='*70}")
    print(f"\n🎯 NLP FEATURES ENABLED:")
    print(f"   ✓ Tokenization (NLTK)")
    print(f"   ✓ POS Tagging - Nouns, Verbs, Adjectives")
    print(f"   ✓ Lemmatization (WordNet)")
    print(f"   ✓ Stopword Removal")
    print(f"   ✓ Sentiment Analysis (TextBlob)")
    print(f"   ✓ Keyword Extraction")
    print(f"\n🌍 SUPPORTED LANGUAGES:")
    print(f"   English, Hindi, Bengali, Tamil, Telugu, Malayalam,")
    print(f"   Marathi, Gujarati, Kannada, Punjabi, Odia, Assamese, Urdu")
    print(f"\n📋 COPY THIS URL:")
    print(f"   {clean_url}")
    print(f"{'='*70}")
    print(f"\n🧪 Test endpoints:")
    print(f"   • {clean_url}/")
    print(f"   • {clean_url}/api/languages")
    print(f"   • {clean_url}/api/attractions")
    print(f"\n💡 Watch console for detailed NLP logs!")
    print(f"\n⚠️  Keep this cell running!")
    print(f"{'='*70}\n")

    # Run the Flask app
    app.run(host='0.0.0.0', port=5000, debug=False)
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\n" + "="*70)
    print("🔧 TROUBLESHOOTING:")
    print("="*70)
    print("\n1. Go to: https://dashboard.ngrok.com/tunnels/agents")
    print("2. Stop all tunnels")
    print("3. Run this cell again")
    print("\nOR Runtime → Restart runtime and run all cells")
    print("="*70)

🔄 Cleaning up...
🚀 Starting NLTK-enhanced server...

✅ NLTK-POWERED TRAVEL PLANNER IS LIVE!

🎯 NLP FEATURES ENABLED:
   ✓ Tokenization (NLTK)
   ✓ POS Tagging - Nouns, Verbs, Adjectives
   ✓ Lemmatization (WordNet)
   ✓ Stopword Removal
   ✓ Sentiment Analysis (TextBlob)
   ✓ Keyword Extraction

🌍 SUPPORTED LANGUAGES:
   English, Hindi, Bengali, Tamil, Telugu, Malayalam,
   Marathi, Gujarati, Kannada, Punjabi, Odia, Assamese, Urdu

📋 COPY THIS URL:
   https://thymy-nonincarnate-jax.ngrok-free.dev

🧪 Test endpoints:
   • https://thymy-nonincarnate-jax.ngrok-free.dev/
   • https://thymy-nonincarnate-jax.ngrok-free.dev/api/languages
   • https://thymy-nonincarnate-jax.ngrok-free.dev/api/attractions

💡 Watch console for detailed NLP logs!

⚠️  Keep this cell running!

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:16:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:16:47] "OPTIONS / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:16:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:16:56] "OPTIONS /api/users HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:16:57] "POST /api/users HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:17:03] "OPTIONS /api/groups HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:17:04] "POST /api/groups HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:17:32] "OPTIONS /api/preferences HTTP/1.1" 200 -



🔍 NLTK-POWERED NLP PROCESSING
🌐 Language detected: mr - Marathi (मराठी)
🔄 Translation:
   Original (mr): आम्ही कुटुंबासह ब्राझिलला जाऊ इच्छितो. ५ दिवसांसाठी समुद्रकिनारे, उद्याने आणि मुलांसाठी मनोरंजन हवे आहे. आमचे बजेट ५०००० रुपये आहे. आम्हाला सुंदर निसर्ग आणि सुरक्षित ठिकाणे पाहायची आहेत.
   English: We want to go to Brazil with family. 5 days worth of beaches, parks and entertainment for kids. Our budget is Rs.50000. We want to see beautiful nature and safe places.
📝 NLTK Tokens (first 15): ['we', 'want', 'to', 'go', 'to', 'brazil', 'with', 'family', '.', '5', 'days', 'worth', 'of', 'beaches', ',']
🧹 Stopword removal: 36 → 18 tokens
🔤 Lemmatized (first 10): ['want', 'go', 'brazil', 'family', '5', 'day', 'worth', 'beach', 'park', 'entertainment']
🏷️ POS Analysis:
   Nouns: ['Brazil', 'family', 'days', 'beaches', 'parks', 'entertainment', 'kids', 'budget']
   Verbs: ['want', 'go', 'is', 'want', 'see']
   Adjectives: ['beautiful', 'safe']
😊 Sentiment: Positive 😊 (score: 0.55)
🏷️ Categ

INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:17:37] "POST /api/preferences HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:18:15] "OPTIONS /api/groups/1/itinerary HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:18:15] "POST /api/groups/1/itinerary HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:23:55] "OPTIONS /api/users HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:23:55] "POST /api/users HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:24:13] "OPTIONS /api/preferences HTTP/1.1" 200 -



🔍 NLTK-POWERED NLP PROCESSING
🌐 Language detected: ta - Tamil (தமிழ்)


...

INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:24:41] "OPTIONS /api/groups/1/itinerary HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:24:41] "POST /api/groups/1/itinerary HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:27:30] "OPTIONS / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:27:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:27:42] "OPTIONS /api/users HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:27:42] "POST /api/users HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:00] "OPTIONS /api/groups HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:01] "POST /api/groups HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:23] "OPTIONS /api/preferences HTTP/1.1" 200 -



🔍 NLTK-POWERED NLP PROCESSING
🌐 Language detected: mr - Marathi (मराठी)


INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:24] "POST /api/preferences HTTP/1.1" 201 -


🔄 Translation:
   Original (mr): आम्ही हनीमूनसाठी उत्तराखंडला जाऊ इच्छितो. ४ दिवसांसाठी रोमँटिक हिल स्टेशन्स, शांत सरोवरे आणि लक्झरी रिसॉर्ट्स हवे आहेत. आमचे बजेट ७०००० रुपये आहे. नैनीताल आणि मसूरीमध्ये सुंदर निसर्ग अनुभवायचा आहे.
   English: We want to go to Uttarakhand for honeymoon. Romantic hill stations, serene lakes and luxury resorts are all you need for 4 days. Our budget is Rs.70000. Beautiful nature is to be experienced in Nainital and Mussoorie.
📝 NLTK Tokens (first 15): ['we', 'want', 'to', 'go', 'to', 'uttarakhand', 'for', 'honeymoon', '.', 'romantic', 'hill', 'stations', ',', 'serene', 'lakes']
🧹 Stopword removal: 42 → 20 tokens
🔤 Lemmatized (first 10): ['want', 'go', 'uttarakhand', 'honeymoon', 'romantic', 'hill', 'station', 'serene', 'lake', 'luxury']
🏷️ POS Analysis:
   Nouns: ['Uttarakhand', 'honeymoon', 'Romantic', 'hill', 'stations', 'lakes', 'luxury', 'resorts']
   Verbs: ['want', 'go', 'are', 'need', 'is']
   Adjectives: ['serene']
😊 Sentiment: Positive 😊 (score: 

INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:58] "OPTIONS /api/groups/2/itinerary HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2025 06:28:58] "POST /api/groups/2/itinerary HTTP/1.1" 201 -


In [ ]:

# Example 1: English
print("\n" + "="*50)
print("TEST 1: English Processing")
print("="*50)
test_en = "I want to visit beautiful temples and amazing beaches for 5 days with a budget of 20000"
lang = lang_processor.detect_language(test_en)
tokens = lang_processor.tokenize_text(test_en)
filtered = lang_processor.remove_stopwords(tokens)
lemmatized = lang_processor.lemmatize_tokens(filtered)
pos = lang_processor.extract_pos_tags(test_en)
sentiment = lang_processor.analyze_sentiment(test_en)
categories = pref_extractor.extract_categories(test_en, lemmatized)
budget = pref_extractor.extract_budget(test_en)
duration = pref_extractor.extract_duration(test_en)

print(f"\n✅ Results:")
print(f"Language: {lang}")
print(f"Tokens: {tokens[:10]}")
print(f"Lemmatized: {lemmatized[:10]}")
print(f"Nouns: {pos['nouns'][:5]}")
print(f"Verbs: {pos['verbs'][:5]}")
print(f"Sentiment: {sentiment}")
print(f"Categories: {categories}")
print(f"Budget: {budget}")
print(f"Duration: {duration}")

# Example 2: Hindi
print("\n" + "="*50)
print("TEST 2: Hindi Processing")
print("="*50)
test_hi = "मैं 5 दिनों के लिए सुंदर मंदिर और समुद्र तट देखना चाहता हूं"
lang_hi = lang_processor.detect_language(test_hi)
translated = lang_processor.translate_to_english(test_hi, lang_hi)
print(f"\n✅ Translation successful!")
print(f"Original: {test_hi}")
print(f"Translated: {translated}")

# Example 3: Tamil
print("\n" + "="*50)
print("TEST 3: Tamil Processing")
print("="*50)
test_ta = "நான் 5 நாட்களுக்கு கோவில்கள் மற்றும் கடற்கரைகளை பார்க்க விரும்புகிறேன்"
lang_ta = lang_processor.detect_language(test_ta)
translated_ta = lang_processor.translate_to_english(test_ta, lang_ta)
print(f"\n✅ Translation successful!")
print(f"Original: {test_ta}")
print(f"Translated: {translated_ta}")

print("\n" + "="*50)
print("✅ ALL TESTS PASSED!")
print("="*50)


